# Device Vendor Classifier

## Get nPrint-wlan output

In [2]:
filename = 'wlan_2020_11_05_03'
# cmd_test = f'nprint -w -P {filename}.pcap -W {filename}.npt'
cmd_test = f'../nprint -w -P {filename}.pcap -W {filename}.npt'
!{cmd_test}

## Print

In [3]:
import pandas as pd

nprint_wlan = pd.read_csv(f'{filename}.npt', index_col=0)

print('nPrint_wlan: Number of Packets: {0}, Features per packet: {1}'.format(nprint_wlan.shape[0], nprint_wlan.shape[1]))
nprint_wlan

nPrint_wlan: Number of Packets: 4422, Features per packet: 528


,radiotap_reversion_0,radiotap_reversion_1,radiotap_reversion_2,radiotap_reversion_3,radiotap_reversion_4,radiotap_reversion_5,radiotap_reversion_6,radiotap_reversion_7,radiotap_pad_0,radiotap_pad_1,...,wlan_rx_addr_38,wlan_rx_addr_39,wlan_rx_addr_40,wlan_rx_addr_41,wlan_rx_addr_42,wlan_rx_addr_43,wlan_rx_addr_44,wlan_rx_addr_45,wlan_rx_addr_46,wlan_rx_addr_47
src_mac,,,,,,,,,,,,,,,,,,,,,
f8:bc:0e:53:3d:a9,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
f8:bc:0e:53:3d:a3,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
f8:bc:0e:53:3d:a6,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
f8:bc:0e:53:3d:a9,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
f8:bc:0e:53:3d:a3,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cc:f4:11:29:f8:89,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
82:f2:9e:2f:34:d8,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
bc:ff:eb:f2:b7:88,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1


## Drop samples without src_mac 

In [9]:
nprint_wlan_with_src_mac = nprint_wlan.loc[nprint_wlan.index != 'None']
print('nprint_wlan_with_src_mac: Number of Packets: {0}, Features per packet: {1}'.format(nprint_wlan_with_src_mac.shape[0], nprint_wlan_with_src_mac.shape[1]))

num_sample_packets = nprint_wlan_with_src_mac.shape[0]

nprint_wlan_with_src_mac: Number of Packets: 3910, Features per packet: 528


## MAC OUI to Vendor

In [5]:
vender_map = {} # key: mac, '00:00:01', value: ('short name', 'long name')
manuf_path = 'MAC_OUI/manuf.txt'

def load_vender_map():
    with open(manuf_path) as f:
        for line in f:
            line = line.strip()
            if not line: continue
            if line.startswith('#'): continue
            if '#' in line: line = line[:line.index('#')].strip()
            
            lsp = list(filter(None, line.split('\t')))
            assert 2 <= len(lsp) <= 3, f'{line}, {lsp}'

            if len(lsp) == 2: mac, sname, lname = lsp[0], lsp[1], lsp[1]
            else: mac, sname, lname = lsp

            if mac.endswith('/28'): # eample: 00:55:DA:00:00:00/28
                mac = mac[:10]
            
            vender_map[mac] = (sname, lname)

load_vender_map()

In [6]:
def get_vendor(mac: str):
    mac = mac.upper()
    if mac[:8] in vender_map.keys():
        return vender_map[mac[:8]][0]
    if mac[:10] in vender_map.keys():
        return vender_map[map[:10]][0]
    return 'None'

## List all vendors

In [13]:
from collections import Counter

c = Counter([get_vendor(row.name) for _, row in nprint_wlan_with_src_mac.iterrows()])

c.most_common()



[('None', 769),
 ('Motorola', 700),
 ('eero', 560),
 ('ARRISGro', 425),
 ('2Wire', 418),
 ('Google', 315),
 ('HewlettP', 213),
 ('ASUSTekC', 170),
 ('Pegatron', 89),
 ('Tp-LinkT', 86),
 ('BelkinIn', 46),
 ('HonHaiPr', 40),
 ('Chongqin', 20),
 ('Netgear', 19),
 ('Shenzhen', 16),
 ('Technico', 7),
 ('IntelCor', 5),
 ('Apple', 5),
 ('OnePlusT', 2),
 ('Sonos', 2),
 ('AmazonTe', 2),
 ('TexasIns', 1)]

## Samples and Lables

- Samples: nprint_wlan_with_src_mac
- Lables: if the src_mac belongs to Apple

In [70]:
import numpy as np

samples = []
labels = []
# positive_vendor = 'Apple'
positive_vendor = '  '

none_samples = []

for _, row in nprint_wlan_with_src_mac.iterrows():
    vendor = get_vendor(row.name)
    if vendor == 'None':  # drop samples without a vendor assigned
        continue
    labels.append(positive_vendor) if vendor == positive_vendor else labels.append('Others')
    samples.append(np.array(row))

print(f'{len(samples)}, {len(samples[0])}')
num_positive = len([True for vendor in labels if vendor == positive_vendor])
print(f'{positive_vendor}: {num_positive}')

3141, 528
Google: 315


## Training

In [71]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

# Split data
X_train, X_test, y_train, y_test = train_test_split(samples, labels)

# Initialize Classifier
clf = RandomForestClassifier(n_estimators=1000, max_depth=None, min_samples_split=2, random_state=0)

# Train 
clf.fit(X_train, y_train) 

# Predict
y_pred = clf.predict(X_test)

# Statistics

# First, lets get a stat report about the precision and recall:
report = classification_report(y_test, y_pred)
print(report)

# Let's also get the ROC AUC score while we're here, which requires a probability instead of just the prediction
y_pred_proba = clf.predict_proba(X_test)
# predict_proba gives us a probability estimate of each class, while roc_auc just cares about the "positive" class
y_pred_proba_pos = [sublist[1] for sublist in y_pred_proba]
roc = roc_auc_score(y_test, y_pred_proba_pos)
print('ROC AUC Score: {0}'.format(roc))

              precision    recall  f1-score   support

      Google       0.99      0.88      0.93        89
      Others       0.98      1.00      0.99       697

    accuracy                           0.98       786
   macro avg       0.99      0.94      0.96       786
weighted avg       0.98      0.98      0.98       786

ROC AUC Score: 0.9988070865507068


## If we use another dataset
- test = 'wlan_2020_11_05_02'


In [72]:
test_file = 'wlan_2020_11_05_02'
cmd_test = f'../nprint -w -P {test_file}.pcap -W {test_file}.npt'
!{cmd_test}

nprint_wlan_test = pd.read_csv(f'{test_file}.npt', index_col=0)
print('nPrint_wlan: Number of Packets: {0}, Features per packet: {1}'.format(nprint_wlan_test.shape[0], nprint_wlan_test.shape[1]))

# Drop samples without src_mac
nprint_wlan_test_with_src_mac = nprint_wlan_test.loc[nprint_wlan_test.index != 'None']

c = Counter([get_vendor(row.name) for _, row in nprint_wlan_test_with_src_mac.iterrows()])
c.most_common()

nPrint_wlan: Number of Packets: 4782, Features per packet: 528


[('eero', 3604),
 ('HewlettP', 633),
 ('Google', 470),
 ('None', 24),
 ('Chongqin', 14),
 ('IntelCor', 6),
 ('Apple', 2),
 ('TexasIns', 1)]

In [73]:
test_samples = []
test_labels = []
# positive_vendor = 'Apple'
# positive_vendor = 'Google'

for _, row in nprint_wlan_test_with_src_mac.iterrows():
    vendor = get_vendor(row.name)
    if vendor == 'None': 
        continue
    test_labels.append(positive_vendor) if vendor == positive_vendor else test_labels.append('Others')
    test_samples.append(np.array(row))

print(f'samples: {len(test_samples)}, {len(test_samples[0])}, lables: {len(test_labels)}')
num_positive = len([True for vendor in test_labels if vendor == positive_vendor])
print(f'{positive_vendor}: {num_positive}')

samples: 4730, 528, lables: 4730
Google: 470


In [75]:
X_test = test_samples
y_test = test_labels

# Predict
y_pred = clf.predict(X_test)

# Statistics

# First, lets get a stat report about the precision and recall:
report = classification_report(y_test, y_pred)
print(report)

# Let's also get the ROC AUC score while we're here, which requires a probability instead of just the prediction
y_pred_proba = clf.predict_proba(X_test)
# predict_proba gives us a probability estimate of each class, while roc_auc just cares about the "positive" class
y_pred_proba_pos = [sublist[1] for sublist in y_pred_proba]
roc = roc_auc_score(y_test, y_pred_proba_pos)
print('ROC AUC Score: {0}'.format(roc))

              precision    recall  f1-score   support

      Google       0.83      0.83      0.83       470
      Others       0.98      0.98      0.98      4260

    accuracy                           0.97      4730
   macro avg       0.91      0.91      0.91      4730
weighted avg       0.97      0.97      0.97      4730

ROC AUC Score: 0.9858985116371991


## Create labels file for nPrintML

In [81]:
labels_path = 'vendor-labels.txt'

positive_vendor = 'Google'
labels_map = {}

for mac in nprint_wlan.index:
    if mac == 'None': labels_map[mac] = 'None'
    labels_map[mac] = 'Google' if get_vendor(mac) == 'Google' else 'Others'

with open(labels_path, 'w') as f:
    f.write('item,label\n')
    for k, v in labels_map.items():
        f.write(f'{k},{v}\n')

## Execute nPrintML

In [86]:
pcap_path = 'wlan_2020_11_05_03.pcap'

nml_cmd = f'nprintml --wlan --pcap-file {pcap_path} --label-file {labels_path} --aggregator index'
!{nml_cmd}

/home/kaiyu/miniconda3/envs/nprint/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[warn] nprint expected version for nprintML (1.1.6) does not match version on PATH (1.0.4 at /usr/local/bin/nprint)
step:Net → NetResult(nprint_path=PosixPath('nprintml/run-bison-1614288234-19396/nprint'))
Loading nPrint: nprintml/run-bison-1614288234-19396/nprint/wlan_2020_11_05_03.npt
Loaded 1 nprint
  nPrint shape: (4422, 528)
Loading labels: vendor-labels.txt
  number of labels: 92
Attaching labels to nPrints
  labels attached: missing labels for:
    missing labels caused samples to be dropped: 0
step:Label → LabelResult(features=                   radiotap_reversion_0  ...   label
06:7f:2c:c3:10:90           

## Try AutoGluon Directly

In [87]:
import autogluon as ag
from autogluon import TabularPrediction as task


# tra = pd.read_csv(f'{filename}.npt', index_col=0)

# # Split data
# X_train, X_test, y_train, y_test = train_test_split(samples, labels)

# output_dir = 'autogluon'
# predictor = task.fit(train_data=X_train, label=y_train, output_directory=dir)